In [145]:
import json

import pandas as pd
import altair as alt

In [146]:
mit = pd.read_csv("Mit UI.csv")
ohne = pd.read_csv("Ohne UI.csv")
demo = pd.read_csv("Demografische Daten.csv")

In [147]:
mit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 35 columns):
 #   Column                                                                                         Non-Null Count  Dtype
---  ------                                                                                         --------------  -----
 0   Kürzel                                                                                         10 non-null     int64
 1   Es war einfach zu verstehen, wie Aktionen im Spiel auszuführen sind.                           10 non-null     int64
 2   Die Aktionen zur Spielsteuerung waren für mich klar                                            10 non-null     int64
 3   Ich fand das Spiel war leicht zu bedienen                                                      10 non-null     int64
 4   Ich habe das Gesamtziel des Spiels verstanden                                                  10 non-null     int64
 5   Die Ziele des Spiels waren für mich kla

In [148]:
long_name = mit.columns[1]

temp_df = mit[[long_name]].copy()
temp_df.columns = ['score']
chart = alt.Chart(temp_df).mark_boxplot(extent='min-max').encode(
    y=alt.Y('score:Q',
            scale=alt.Scale(domain=[3, 6]), # Force scale to show 1-6
            title="Antwort-Skala")
).properties(
    width=200,
    title=f"{long_name}..."
)

chart.display()


alt.Chart(...)

In [149]:
# 1. Select the columns you want to compare (e.g., index 1 to 6)
# We use .iloc to select by index position
cols_to_compare = mit.columns[1:]
subset = mit[cols_to_compare]

# 2. "Melt" the data: This turns columns into rows
# 'var_name' becomes the new column for the Questions
# 'value_name' becomes the new column for the 1-6 Scores
melted_df = subset.melt(var_name='Frage', value_name='Score')

# 3. Create the multi-column Boxplot
chart = alt.Chart(melted_df).mark_boxplot(extent='min-max').encode(
    x=alt.X('Frage:N',
            axis=alt.Axis(labels=False), # Hides the long sentences on the X-axis
            title="Fragen (Vergleich)"),
    y=alt.Y('Score:Q',
            scale=alt.Scale(domain=[0, 6]),
            title="Antwort-Skala (1-6)"),
    color=alt.Color('Frage:N', legend=alt.Legend(title="Fragen-Index", labelLimit=300)),
    tooltip=['Frage', 'Score'] # Hover to see the full question and value
).properties(
    width=500,
    height=400,
    title="Vergleich der Spielerfahrung"
)

chart.display()

alt.Chart(...)

In [150]:
def aggregate_by_catergory(df):# Define your categories
    categories = [
        "EaseOfControl", "GoalsAndRules", "Challange", "ProgressFeedback",
        "AudiovisualAppeal", "Meaning", "Curiosity", "Mastery",
        "Immersion", "Autonomie", "UI"
    ]

    # Create a new empty DataFrame
    df_aggregated = pd.DataFrame()

    # Loop through and average every 3 columns
    for i, cat in enumerate(categories):
        start_col = (i * 3) + 1  # +1 because index 0 is 'Kürzel'

        if cat == "UI":
            # For the last category, we take everything from 31 to the end
            df_aggregated[cat] = df.iloc[:, start_col:].mean(axis=1)
        else:
            # Take exactly 3 columns and average them
            end_col = start_col + 3
            df_aggregated[cat] = df.iloc[:, start_col:end_col].mean(axis=1)

    # Resulting dataframe has 6 rows (one per participant) and 11 category columns
    return df_aggregated

In [151]:
df_aggregated_mit = aggregate_by_catergory(mit)
df_aggregated_ohne = aggregate_by_catergory(ohne)

In [152]:
def visualize_boxplotted(df):
    cols_to_compare = df.columns[0:]
    subset = df[cols_to_compare]
    melted_df = subset.melt(var_name='Frage', value_name='Score')

    # 1. Base Chart: Define the common data and encoding
    base = alt.Chart(melted_df).encode(
        x=alt.X('Frage:N', axis=alt.Axis(labels=False), title="Fragen (Vergleich)"),
        color=alt.Color('Frage:N', legend=alt.Legend(title="Fragen-Index", labelLimit=300))
    )

    # 2. Boxplot Layer
    boxplot = base.mark_boxplot(extent='min-max').encode(
        y=alt.Y('Score:Q', scale=alt.Scale(domain=[0, 6]), title="Antwort-Skala (1-6)"))

    # 3. Mean Layer: Use 'mean(Score)' to let Altair do the math
    mean_points = base.mark_point(
        shape='cross',
        size=50,
        filled=True
    ).encode(
        y='mean(Score):Q',
        color=alt.value('black'),
        tooltip=[
            alt.Tooltip('Frage:N', title='Frage'),
            alt.Tooltip('mean(Score):Q', title='Durchschnitt', format='.2f') # Rounds to 2 decimal places
        ]
    )

    # 4. Combine them using the + operator
    chart = (boxplot + mean_points).properties(
        width=500,
        height=400,
        title="Vergleich der Spielerfahrung"
    )

    chart.display()


In [153]:
visualize_boxplotted(df_aggregated_mit)

alt.LayerChart(...)

In [154]:
visualize_boxplotted(df_aggregated_ohne)

alt.LayerChart(...)

In [155]:
import os
import re
import json
from collections import defaultdict

def get_grouped_stats_data(root_dir):
    """
    Returns a dictionary grouped by {participant_id: {category: [sorted_file_contents]}}
    Files are sorted by the 'playTime' field inside the JSON.
    """
    # Temporary storage for filtered "latest" files
    # Structure: (p_id, category, content_dict)
    all_selected_data = []

    for root, dirs, files in os.walk(root_dir):
        folder_name = os.path.basename(root)
        if not ('_mit' in folder_name or '_ohne' in folder_name):
            continue

        p_id, category = folder_name.split('_')

        # 1. Standardize to find the latest stats_<name>_<number>
        for filename in files:
            file_path = os.path.join(root, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    all_selected_data.append({
                        'p_id': p_id,
                        'category': category,
                        'data': data,
                        # 'playTime': data.get('playTime', 0) # Use 0 if missing
                    })
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

    # 3. Sort everything by playTime first
    # all_selected_data.sort(key=lambda x: x['playTime'])

    # 4. Group by Participant ID and then Category
    grouped_results = defaultdict(lambda: defaultdict(list))

    for entry in all_selected_data:
        # We store the data back as a JSON string to match your requirement
        content_string = json.dumps(entry['data'])
        grouped_results[entry['p_id']][entry['category']].append(entry['data'])


    for (key1, group) in grouped_results.items():
        for (key2, value) in group.items():
            grouped_results[key1][key2].sort(key=lambda x: x['playTime'])

    for (key, value) in grouped_results.items():
        for (inner_key, inner_value) in value.items():
            if inner_key == 'mit' or inner_key == 'ohne':
                new_list = []
                last_room = None
                last_obj = None
                for entry in inner_value:
                    if last_room != entry["sceneName"] and last_obj is not None:
                        new_list.append(last_obj)
                    last_room = entry["sceneName"]
                    last_obj = entry
                if last_obj is not None:
                    new_list.append(last_obj)

                value[inner_key] = new_list

    return dict(grouped_results)

# --- Usage ---
# final_data = get_grouped_stats_data('./data')
# Example: Accessing participant '053332' in the 'mit' category:
# list_of_json_strings = final_data['053332']['mit']


In [156]:

data_tuples = get_grouped_stats_data("../participant_runs")
print(data_tuples)
#data_json = []
#for p_id, cat, content in data_tuples:
#    obj = json.loads(content)
#    data_json.append({"id": p_id, "category": cat, "content": obj})
#
#
with open("parsed_files.json", "w") as f:
    json.dump(data_tuples, f, indent=2)

{'071225': defaultdict(<class 'list'>, {'mit': [{'sceneName': 'Tutorial', 'hits': 1, 'damageTaken': 3, 'deaths': 1, 'playTime': 35.27436447143555}, {'sceneName': 'Lobby', 'hits': 1, 'damageTaken': 3, 'deaths': 1, 'playTime': 51.636260986328125}, {'sceneName': 'Treppenhaus', 'hits': 15, 'damageTaken': 17, 'deaths': 18, 'playTime': 236.5513458251953}, {'sceneName': 'Lobby', 'hits': 15, 'damageTaken': 17, 'deaths': 18, 'playTime': 255.31231689453125}, {'sceneName': 'StorageRoom', 'hits': 17, 'damageTaken': 19, 'deaths': 32, 'playTime': 419.2154846191406}, {'sceneName': 'Lobby', 'hits': 17, 'damageTaken': 19, 'deaths': 32, 'playTime': 433.81640625}, {'sceneName': 'Treppenhaus', 'hits': 17, 'damageTaken': 19, 'deaths': 32, 'playTime': 436.3377380371094}, {'sceneName': 'Lobby', 'hits': 17, 'damageTaken': 19, 'deaths': 32, 'playTime': 439.7655334472656}, {'sceneName': 'Boss', 'hits': 34, 'damageTaken': 36, 'deaths': 38, 'playTime': 807.9039306640625}], 'ohne': [{'sceneName': 'Tutorial', 'hits